In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import concat

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1651950817854_0005,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession\
    .builder \
    .appName("Kmeans Clustering Lab") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are reading data from the CSV file and putting it into a Spark dataframe and letting it infer the schema from the header.

In [24]:
df = spark.read.csv('s3://cs496-proga1-storage/covid_data/metadata.csv',
                    sep=',',
                    inferSchema=True,
                    header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cord_uid: string (nullable = true)
 |-- sha: string (nullable = true)
 |-- source_x: string (nullable = true)
 |-- title: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- pmcid: string (nullable = true)
 |-- pubmed_id: string (nullable = true)
 |-- license: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- mag_id: string (nullable = true)
 |-- who_covidence_id: string (nullable = true)
 |-- arxiv_id: string (nullable = true)
 |-- pdf_json_files: string (nullable = true)
 |-- pmc_json_files: string (nullable = true)
 |-- url: string (nullable = true)
 |-- s2_id: string (nullable = true)

We only want to keep the tile and abstract columns. We drop the rest from the dataframe.

In [26]:
df = df.drop('cord_uid', 'sha', 'source_x', 'doi', 'pmcid', 'pubmed_id', 'license', 'publish_time', 'authors', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files', 'url', 's2_id')
# Display the dataframe.
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|               title|            abstract|
+--------------------+--------------------+
|Clinical features...|OBJECTIVE: This r...|
|Nitric oxide: a p...|Inflammatory dise...|
|Surfactant protei...|Surfactant protei...|
|Role of endotheli...|Endothelin-1 (ET-...|
|Gene expression i...|Respiratory syncy...|
|Sequence requirem...|Nidovirus subgeno...|
|Debate: Transfusi...|Recent evidence s...|
|The 21st Internat...|The 21st Internat...|
|Heme oxygenase-1 ...|Heme oxygenase-1 ...|
|Technical Descrip...|This report descr...|
|Conservation of p...|Regulation of orn...|
|Heterogeneous nuc...|Heterogeneous nuc...|
|A Method to Ident...|The UBA domain is...|
|Vaccinia virus in...|We examined the r...|
|The site of origi...|                null|
|Multi-faceted, mu...|There are hundred...|
|Herpes simplex vi...|INTRODUCTION: The...|
|Logistics of comm...|BACKGROUND: Previ...|
|Protection of pul...|BACKGROUND: Oxyge...|
|Bioinformatic map...|BACKGROUND

In [31]:
# We drop the null values from the dataframe.
df = df.na.drop()
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|               title|            abstract|
+--------------------+--------------------+
|Clinical features...|OBJECTIVE: This r...|
|Nitric oxide: a p...|Inflammatory dise...|
|Surfactant protei...|Surfactant protei...|
|Role of endotheli...|Endothelin-1 (ET-...|
|Gene expression i...|Respiratory syncy...|
|Sequence requirem...|Nidovirus subgeno...|
|Debate: Transfusi...|Recent evidence s...|
|The 21st Internat...|The 21st Internat...|
|Heme oxygenase-1 ...|Heme oxygenase-1 ...|
|Technical Descrip...|This report descr...|
|Conservation of p...|Regulation of orn...|
|Heterogeneous nuc...|Heterogeneous nuc...|
|A Method to Ident...|The UBA domain is...|
|Vaccinia virus in...|We examined the r...|
|Multi-faceted, mu...|There are hundred...|
|Herpes simplex vi...|INTRODUCTION: The...|
|Logistics of comm...|BACKGROUND: Previ...|
|Protection of pul...|BACKGROUND: Oxyge...|
|Bioinformatic map...|BACKGROUND: AlkB-...|
|Managing emerging...|In the 198

In [40]:
# We concatenate the columns into one.
dframe = df.select(concat(df.title,df.abstract))
dframe = dframe.withColumnRenamed("concat(title, abstract)","data")
dframe.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
# What is the schema of the new datafrome?
dframe.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- data: string (nullable = true)

In [42]:
from pyspark.sql.functions import regexp_replace
# We strip all the whitespaces in the data column.
dframe = dframe.withColumn('DATA', regexp_replace(col("data"), " ", ""))
dframe.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------